### Input

detected pattern patternlist.xlsx

tagged named entities (chemical, gene and fuzzy"O") train0_truth.tsv

### Output

confidence of patterns and named entities

Then evaluate them by the matching them to train1.ner.txt

### data location

tagged file from fuzzy-ner: /shared/data/shizhi2/data_fuzzyCRF/case/train0_all.tsv, train1_all.tsv (edited) **train0_gene.tsv**, **train1_chem.tsv**

ground truth: /shared/data/shizhi2/data_fuzzyCRF/bio-ner/**train0_all.tsv**, **train1_all.tsv** contain all the gene and chemical labels

patternlist: **patternlist.xlsx**

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/flyingturtle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

S-GENE single

B-GENE begining of the gene

E-GENE end of the gene

In [13]:
named_entity_list = {}
with open("train1_all.tsv", "r") as f:
    for line in f:
        segs = line[:-1].split("\t")
        named_entity = " ".join(segs[:-1])
        entity_label = segs[-1]
        
        # check if there is a collision
        try:
            if(named_entity_list[named_entity]):
                pass
        except:
            named_entity_list[named_entity] = [entity_label]
            
        if(entity_label != "O" and entity_label not in named_entity_list[named_entity]):
            named_entity_list[named_entity].append(entity_label)

print("number of named entities with labels: ", len(named_entity_list.keys()))
print(named_entity_list)

number of named entities with labels:  43135
{'DPP6': ['O', 'S-GENE', 'B-GENE'], 'as': ['O', 'S-Chemical'], 'a': ['O', 'I-Chemical', 'E-Chemical', 'I-GENE', 'E-GENE'], 'candidate': ['O'], 'gene': ['O', 'E-GENE', 'I-GENE', 'B-GENE'], 'for': ['O'], 'neuroleptic': ['O'], '=-': ['O', 'I-GENE', 'I-Chemical', 'B-Chemical', 'B-GENE', 'E-Chemical'], 'induced': ['O', 'I-GENE'], 'tardive': ['O'], 'dyskinesia': ['O'], '.': ['O', 'I-Chemical', 'I-GENE', 'E-Chemical'], '': ['', 'I-Chemical', 'I-GENE', 'E-Chemical'], 'We': ['O'], 'implemented': ['O'], 'two': ['O'], 'step': ['O'], 'approach': ['O'], 'to': ['O', 'I-Chemical', 'I-GENE'], 'detect': ['O'], 'potential': ['O', 'I-GENE', 'E-GENE'], 'predictor': ['O'], 'variants': ['O', 'E-GENE'], '(': ['O', 'I-Chemical', 'B-Chemical', 'I-GENE', 'B-GENE'], 'TD': ['O'], ')': ['O', 'I-Chemical', 'E-Chemical', 'I-GENE', 'E-GENE'], 'in': ['O', 'I-GENE'], 'schizophrenic': ['O'], 'subjects': ['O'], 'First': ['O'], 'we': ['O'], 'screened': ['O'], 'associations': ['

In [ ]:
print(named_entity_list[:10])

In [13]:
pattern_list = pd.read_excel("patternlist.xlsx", header=None, names=["id", "pattern"])
print(pattern_list.head())
print(len(pattern_list))

            id                           pattern
0    PATTERN10                  CHEMICAL therapy
1   PATTERN100  transcriptional activity of GENE
2  PATTERN1001               100 microm CHEMICAL
3  PATTERN1003               activity in DISEASE
4  PATTERN1005             the DISEASE cell line
4780


In [ ]:
with open("train1.ner.txt") as f:
    whole_context = f.read()
    context_segs = whole_context.split("\n")

In [ ]:
context_segs[:5]

In [ ]:
def pmatch(pattern, document):
    """
    This function matchs a pattern containing "CHEMICAL", "GENE", "DISEASE" to a given document
    """
    
    res = {"content": pattern, "matches":[]}
    
    # generate re query for search
    match_query = re.sub('CHEMICAL|GENE|DISEASE', "(.*?)", pattern)
    if(match_query[-5:]=="(.*?)"):
        match_query = match_query[:-5] + "(.*)"
        
    # search pattern in the document
    matches = re.search(match_query, document)
    pos_pattern = list(re.finditer("CHEMICAL|GENE|DISEASE", pattern))
    
    index = 1
    for p in pos_pattern[:-1]:
        res["matches"].append((p.group(0), matches.group(index)))
        index += 1
    tail = matches.group(index).split(" ")
    if(tail[0] in ["the", "a", "in"]):
        last_word = tail[1]
    else:
        last_word = tail[0]
    res["matches"].append((pos_pattern[-1].group(0), last_word))
    
    return res

    
    
pmatch("development of DISEASE such as GENE will DISEASE", "The development of the last proteins such as we will king of all the issue")

In [ ]:
from textblob import TextBlob
txt = """king of all the issue"""
blob = TextBlob(txt)
print(blob.noun_phrases)

In [ ]:
for inex, row in pattern_list.iterrows():
    pattern = row["pattern"]
    print(pattern)
    try: # in case pattern == "nan"
        print(re.split('CHEMICAL|GENE|DISEASE|\n',pattern))
    except:
        pass
    

In [9]:
# re
re.search("%", "wh ehiu ) ().., %,")

<_sre.SRE_Match object; span=(16, 17), match='%'>

In [12]:
with open("train_CRAFT_cnt_N_p.tsv", "r") as f:
    for line in f:
        segs = line.split("\t")
        print(segs)

['$W$$N$', 'GENE', '11849\n']
['$W$ - $W$', 'GENE', '2959\n']
['$W$ - $N$', 'GENE', '2844\n']
['$W$$N$', 'Chemical', '2239\n']
['$W$ - $W$', 'Chemical', '1789\n']
['$W$$N$$W$', 'GENE', '1438\n']
['$W$$N$ $W$', 'GENE', '1339\n']
['$W$ - $W$ $W$', 'GENE', '1118\n']
['$N$ - $W$', 'Chemical', '986\n']
['$W$ - $N$ $W$', 'GENE', '892\n']
['$W$$N$$W$$N$', 'GENE', '691\n']
['$W$ - $N$', 'Chemical', '679\n']
['$W$ $W$$N$', 'GENE', '664\n']
['$W$ ( $N$ )', 'Chemical', '533\n']
['$W$ ( $N$ + )', 'Chemical', '522\n']
['$W$ ( $W$ )', 'Chemical', '406\n']
['$W$ - $W$ $W$ $W$', 'GENE', '383\n']
['( $N$ ) $W$', 'Chemical', '316\n']
['$W$ - $W$ $W$', 'Chemical', '274\n']
['$W$ $N$', 'GENE', '266\n']
['$W$$N$$W$', 'Chemical', '265\n']
['$W$$N$$W$$N$', 'Chemical', '258\n']
['$W$$N$ $W$ $W$', 'GENE', '254\n']
['$W$ $W$ - $W$ $W$', 'GENE', '250\n']
['$W$ ( + )', 'Chemical', '236\n']
['$W$ $N$', 'Chemical', '230\n']
['$W$ $W$$N$', 'Chemical', '194\n']
['$W$ - $N$$W$', 'GENE', '185\n']
['$W$ - $W$ - $W$', 'C

['$W$$N$ ( . - )', 'Chemical', '2\n']
["$N$ ' - $W$ ( $W$ ) - $N$ - ( $W$ - $N$ ( $N$$W$ ) - $W$ ) - $W$ - $W$", 'Chemical', '2\n']
['$W$$N$ ( - * )', 'Chemical', '2\n']
['$W$$N$ ( * - )', 'Chemical', '2\n']
['$N$ - $W$ - $N$ - [ $N$ - ( $N$ - $W$ - $N$ , $N$ - $W$ - $N$ - $W$ ) - $N$$W$ - $W$ - $N$ - $W$ ] - $N$ , $N$ - $W$', 'Chemical', '2\n']
['$W$ $W$ - $W$ $W$ - $W$ $W$', 'Chemical', '2\n']
['$N$ - $W$ - $N$ - $W$ - $W$ - $N$ - $W$', 'Chemical', '2\n']
['[ $W$ - $W$ - $N$ ] - $W$$N$', 'GENE', '2\n']
['$W$ - $N$ $W$ - $W$ - $N$ - $W$ $W$', 'Chemical', '2\n']
['( $W$ ) - $N$ - $W$ - $N$ - $W$ - $N$ - ( $N$ - ( $W$ - $N$ - $W$ ) $W$ ) $W$ $W$', 'Chemical', '2\n']
['$W$ [ $W$ ] [ $N$ , $N$ ] $W$ - $N$ ( $N$$W$ ) - $W$', 'Chemical', '2\n']
['$W$ [ $N$ ] $W$ $W$', 'Chemical', '2\n']
['( $W$ ( - ) ) $N$', 'Chemical', '2\n']
['$W$$N$ ( - ) .', 'Chemical', '2\n']
['$N$ - ( $W$ ) - $N$ - ( $N$ - $W$ ) - $N$ - $W$', 'Chemical', '2\n']
['$N$ $W$ - $W$ - $W$ - $W$ - $W$ - $N$ $W$ , $N$ $W$ - $